In [1]:
I = readlines(open("input"))
map, moves = I[begin:findfirst(==(""), I)-1], reduce(*,I[findfirst(==(""), I)+1:end])
map = only.(mapreduce(permutedims, vcat, split.(map, "")))

50×50 Matrix{Char}:
 '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  …  '#'  '#'  '#'  '#'  '#'  '#'  '#'
 '#'  'O'  '.'  '.'  'O'  '.'  '.'  '.'     '.'  '.'  '.'  '.'  '.'  '#'  '#'
 '#'  '.'  '#'  '.'  'O'  '.'  '.'  '.'     '.'  'O'  '.'  '.'  '.'  'O'  '#'
 '#'  '.'  'O'  'O'  'O'  '.'  '.'  '#'     '.'  '.'  '.'  'O'  'O'  '.'  '#'
 '#'  '.'  'O'  '.'  '.'  '#'  '.'  '.'     '#'  '.'  '.'  '.'  '.'  'O'  '#'
 '#'  '.'  '.'  '.'  '.'  '.'  '#'  '.'  …  'O'  '.'  '.'  '#'  '.'  '#'  '#'
 '#'  '#'  '.'  '.'  '#'  '.'  '.'  '#'     '.'  '.'  '#'  '.'  '.'  '.'  '#'
 '#'  '.'  'O'  'O'  '.'  'O'  '.'  '#'     '.'  '.'  '.'  '.'  '.'  'O'  '#'
 '#'  '.'  '.'  'O'  '#'  '.'  '.'  '#'     '#'  'O'  'O'  'O'  'O'  '.'  '#'
 '#'  '.'  '.'  '.'  'O'  '.'  'O'  '.'     'O'  '.'  '.'  'O'  '.'  'O'  '#'
 ⋮                        ⋮              ⋱            ⋮                   
 '#'  'O'  '.'  '.'  '.'  '.'  '.'  'O'     '#'  '.'  'O'  '.'  'O'  '.'  '#'
 '#'  '.'  '.'  '.'  '.'  'O'  '.'  '.'     '.'

In [2]:
up, right, down, left = CartesianIndex(-1, 0), CartesianIndex(0,1), CartesianIndex(1,0), CartesianIndex(0,-1);
move_dict = Dict('<'=>left, '>'=>right, '^'=>up, 'v'=>down)
moves = (c->move_dict[c]).(collect(moves))

20000-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 0)
 CartesianIndex(0, 1)
 CartesianIndex(0, -1)
 CartesianIndex(1, 0)
 CartesianIndex(0, 1)
 CartesianIndex(0, -1)
 CartesianIndex(1, 0)
 CartesianIndex(0, -1)
 CartesianIndex(0, -1)
 CartesianIndex(1, 0)
 ⋮
 CartesianIndex(-1, 0)
 CartesianIndex(1, 0)
 CartesianIndex(0, 1)
 CartesianIndex(-1, 0)
 CartesianIndex(0, -1)
 CartesianIndex(0, -1)
 CartesianIndex(0, -1)
 CartesianIndex(0, 1)
 CartesianIndex(0, 1)

In [48]:
pos = findfirst(==('@'), map)
map[pos] = '.';

In [49]:
for d in moves
    (map[pos+d] == '#') && continue
    if (map[pos+d] == '.')
        pos += d
    elseif(map[pos+d] == 'O')
        i = 2
        while map[pos+i*d] == 'O'
            i += 1
        end
        (map[pos+i*d] == '#') && continue
        map[pos+d] = '.'
        map[pos+i*d] = 'O'
        pos +=d 
    end
end

In [8]:
gps(i) = 100*(i[1]-1) + (i[2]-1)
sum(gps(i) for i in findall(==('O'), map))

1381454

In [3]:
new_map = Matrix{Char}(undef, size(map,1), 2*size(map,2))
replace_dict = Dict('#'=>['#', '#'], 'O'=>['[',']'], '.'=>['.','.'], '@'=>['@','.'])
for i in CartesianIndices(map)
    new_map[i[1],2*i[2]-1:2*i[2]] = replace_dict[map[i]]
end

In [4]:
pos = findfirst(==('@'), new_map)
new_map[pos] = '.';

In [5]:
function gather_all_pos(start,d)
    all_pos = copy(start)
    curr = start
    while !isempty(curr)
        next = []
        for tpos in curr
            new_map[tpos+d] == '#' && return nothing
            new_map[tpos+d] == '.' && continue
            if new_map[tpos+d] == '[' || new_map[tpos+d] == ']'
                push!(next, tpos+d)
                (new_map[tpos+d] != new_map[tpos]) && push!(next, (new_map[tpos+d] == '[') ? tpos+d+right : tpos+d+left)        
            end
        end
        append!(all_pos, unique(next))
        curr = unique(next)
    end
    return all_pos
end

gather_all_pos (generic function with 1 method)

In [6]:
for d in moves
    (new_map[pos+d] == '#') && continue
    if (new_map[pos+d] == '.')
        pos += d
    elseif(new_map[pos+d] == ']' || new_map[pos+d] == '[')
        if d==right || d==left
            i = 2
            while new_map[pos+i*d] == '[' || new_map[pos+i*d] == ']'
                i += 1
            end
            (new_map[pos+i*d] == '#') && continue
            for k in reverse(2:i)
                new_map[pos+k*d] = new_map[pos+(k-1)*d]
            end
            new_map[pos+d] = '.'
            pos +=d 
        else
            start = (new_map[pos+d] == ']') ? [pos+d, pos+d+left] : [pos+d, pos+d+right]
            all_pos = gather_all_pos(start,d)
            isnothing(all_pos) && continue
            for pos in reverse(all_pos)
                new_map[pos+d] = new_map[pos]
                new_map[pos] = '.'
            end
            pos += d
        end
    end
end

In [198]:
new_map

7×14 Matrix{Char}:
 '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'
 '#'  '#'  '.'  '.'  '.'  '.'  '['  ']'  '.'  '.'  '.'  '.'  '#'  '#'
 '#'  '#'  '.'  '.'  '.'  '['  ']'  '.'  '.'  '.'  '.'  '.'  '#'  '#'
 '#'  '#'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '#'  '#'
 '#'  '#'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '#'  '#'
 '#'  '#'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'  '#'  '#'
 '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'  '#'

In [9]:
sum(gps(i) for i in findall(==('['), new_map))

1392847